In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.gridspec import GridSpec

In [9]:
#load file
def load_data(file_path):
    """Load Excel data and format it for visualization"""
    df = pd.read_excel(file_path)
    
    # Check if needed columns exist
    required_cols = ['CF_desc', 'year', 'year_def_ha']
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Required column '{col}' not found in the Excel file")
    
    return df

#create a directory for saving figures
def create_output_dir(path, dir_name="deforestation_figures"):
    """Create a directory to save the figures"""
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return os.path.join(path, dir_name)

#plot deforestation trend for a single community forest
def plot_forest_trend(forest_data, output_dir):
    """Create and save a plot for a single community forest"""
    forest_name = forest_data['CF_desc'].iloc[0]
    
    #sort by year to ensure proper trend line
    forest_data = forest_data.sort_values('year')
    
    #calculate total deforestation
    total_deforestation = forest_data['year_def_ha'].sum()
    is_deforestation = total_deforestation > 0
    
    #create figure
    fig = plt.figure(figsize=(12, 8))
    gs = GridSpec(3, 1, height_ratios=[0.2, 3, 0.5])
    
    #title subplot
    title_ax = plt.subplot(gs[0])
    title_ax.axis('off')
    title_ax.text(0.5, 0.5, forest_name, 
                  horizontalalignment='center', 
                  verticalalignment='center',
                  fontsize=18, fontweight='bold')
    
    #add total deforestation info
    status = "Deforestation (Forest Loss)" if is_deforestation else "Reforestation (Forest Gain)"
    color = "red" if is_deforestation else "green"
    title_ax.text(0.5, 0, f"Total {status}: {abs(total_deforestation):.2f} ha", 
                  horizontalalignment='center', 
                  verticalalignment='top',
                  fontsize=14, color=color)
    
    #main plot
    ax = plt.subplot(gs[1])
    
    #plot horizontal line at y=0
    ax.axhline(y=0, color='black', linestyle='-', alpha=0.7)
    
    #data points and line
    years = forest_data['year'].values
    deforestation = forest_data['year_def_ha'].values

    ax.scatter(years, deforestation, s=100, c='blue', zorder=5)
    
    #create the line plot
    ax.plot(years, deforestation, 'b-', linewidth=2)
    
    #fill color between line and y=0 (red for deforestation, green for reforestation)
    for i in range(len(years) - 1):
        if deforestation[i] > 0:
            ax.fill_between([years[i], years[i+1]], 
                           [deforestation[i], deforestation[i+1]], 
                           0, 
                           alpha=0.3, 
                           color='red')
        else:
            ax.fill_between([years[i], years[i+1]], 
                           [deforestation[i], deforestation[i+1]], 
                           0, 
                           alpha=0.3, 
                           color='green')
    
    #details
    ax.set_xlabel('Year', fontsize=14)
    ax.set_ylabel('Deforestation (ha)', fontsize=14)
    
    #padding
    max_abs_value = max(abs(forest_data['year_def_ha'].min()), abs(forest_data['year_def_ha'].max()))
    y_limit = max(5, max_abs_value * 1.1)  # At least ±5, or 110% of max value
    ax.set_ylim(-y_limit, y_limit)

    ax.set_xlim(min(years) - 0.5, max(years) + 0.5)
    
    #labels
    for i, (year, value) in enumerate(zip(years, deforestation)):
        va = 'bottom' if value < 0 else 'top'
        offset = -10 if value < 0 else 10
        ax.annotate(f'{value:.2f}', 
                    (year, value), 
                    textcoords="offset points",
                    xytext=(0, offset), 
                    ha='center',
                    va=va,
                    fontsize=11)
    
    #grid lines 
    ax.grid(True, linestyle='--', alpha=0.7)
    
    #footer
    footer_ax = plt.subplot(gs[2])
    footer_ax.axis('off')
    footer_text = ("Interpretation guide:\n"
                  "• Positive values (above zero line): Deforestation (forest loss)\n"
                  "• Negative values (below zero line): Reforestation (forest gain)")
    footer_ax.text(0.5, 0.5, footer_text, 
                  horizontalalignment='center', 
                  verticalalignment='center',
                  fontsize=12)
    
    plt.tight_layout()
    
    # Save figure
    safe_name = "".join([c if c.isalnum() else "_" for c in forest_name])
    plt.savefig(f"{output_dir}/{safe_name}.png", dpi=300, bbox_inches='tight')
    plt.savefig(f"{output_dir}/{safe_name}.pdf", bbox_inches='tight')
    
    # Close the figure to free memory
    plt.close(fig)
    
    return f"{safe_name}.png"

# Create all forest plots
def generate_all_plots(excel_file_path, path):
    """Generate and save plots for all community forests in the data"""
    # Load data
    df = load_data(excel_file_path)
    
    # Create output directory
    output_dir = create_output_dir(path)
    
    # Get unique forest names
    forests = df['CF_desc'].unique()
    
    print(f"Generating plots for {len(forests)} community forests...")
    
    # Create a plot for each forest
    generated_files = []
    for forest_name in forests:
        forest_data = df[df['CF_desc'] == forest_name]
        if not forest_data.empty:
            filename = plot_forest_trend(forest_data, output_dir)
            generated_files.append(filename)
            print(f"Created plot for: {forest_name}")
    
    print(f"All plots saved to '{output_dir}' directory")
    return generated_files

# Create a summary visualization
def create_summary_plot(excel_file_path, output_dir=None):
    """Create a summary visualization of total deforestation across all forests"""
    if output_dir is None:
        output_dir = create_output_dir(output_dir)
    else:
        output_dir = os.path.join(output_dir, "deforestation_figures")
        
    # Load data
    df = load_data(excel_file_path)
    
    # Calculate total deforestation for each forest
    forest_totals = df.groupby('CF_desc')['year_def_ha'].sum().reset_index()
    
    # Sort by total deforestation (descending)
    forest_totals = forest_totals.sort_values('year_def_ha')
    
    # Create figure
    plt.figure(figsize=(14, 10))
    
    # Plot horizontal bars
    bars = plt.barh(forest_totals['CF_desc'], forest_totals['year_def_ha'], height=0.7)
    
    # Color bars based on deforestation (red) or reforestation (green)
    for i, bar in enumerate(bars):
        bar.set_color('red' if forest_totals['year_def_ha'].iloc[i] > 0 else 'green')
    
    # Add a vertical line at x=0
    plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
    
    # Add labels and title
    plt.xlabel('Total Deforestation (ha)', fontsize=14)
    plt.ylabel('Community Forest', fontsize=14)
    plt.title('Total Deforestation by Community Forest (2017-2023)', fontsize=18, pad=20)
    
    # Add value labels
    for i, v in enumerate(forest_totals['year_def_ha']):
        label_pos = v + (0.5 if v < 0 else -0.5)
        ha = 'left' if v < 0 else 'right'
        plt.text(label_pos, i, f"{v:.2f}", va='center', ha=ha)
    
    # Add interpretation note
    plt.figtext(0.5, 0.01, 
                'Positive values (red): Deforestation (forest loss) | Negative values (green): Reforestation (forest gain)',
                ha='center', fontsize=11)
    
    # Adjust layout
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    
    # Save the figure
    plt.savefig(f"{output_dir}/summary_all_forests.png", dpi=300, bbox_inches='tight')
    plt.savefig(f"{output_dir}/summary_all_forests.pdf", bbox_inches='tight')
    plt.close()
    
    print(f"Summary plot saved to '{output_dir}/summary_all_forests.png'")


if __name__ == "__main__":
    excel_file = os.path.dirname(os.getcwd()) + "/Data Outputs/Deforestation_and_Reforestation.xlsx"
    directory = os.path.dirname(os.getcwd()) + "/Data Outputs/"
    
    #individual plots for each forest
    generate_all_plots(excel_file, directory)
    
    #summary plot
    create_summary_plot(excel_file, directory)

Generating plots for 31 community forests...
Created plot for: Sambaulye_CF_shapefiles
Created plot for: Kaloswe_CF_Original_Boundary
Created plot for: Mibobo CFA
Created plot for: Kakoko CF
Created plot for: Lokomwila_CF_shapefiles
Created plot for: Chintasama_Community_Forest
Created plot for: Chintasama_CF
Created plot for: Chibulika CF
Created plot for: Kakoma_Samu_CF_shapefiles
Created plot for: Muponyangala Community Forest
Created plot for: Chisope CF
Created plot for: Mulonga_CF_shapefiles
Created plot for: Lumika Community Forest No. 2
Created plot for: Lumika_CF_shapefiles
Created plot for: Tiyeseko CF
Created plot for: Lukalu_Community_Forest_Revised
Created plot for: Samfu
Created plot for: Mutimba Cfmg
Created plot for: Swiswi CFMG
Created plot for: Kambwa CFMG
Created plot for: Kalwala
Created plot for: Mukotwe CFMG
Created plot for: Lwambazi
Created plot for: Kopeka
Created plot for: Mukubwe CFMG
Created plot for: Kanyembo
Created plot for: Lubulafita
Created plot for: C